# TEST code

fixed number to stochastic conversion & multiply & stochastic to fixed number conversion

## Import

In [1]:
import random, struct, math
import torch
import numpy as np
import torch.nn as nn 
import argparse
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Parser

In [2]:
parser = argparse.ArgumentParser(description='fixed_mac')
parser.add_argument('--full_bits', type=int, default=16, help='Number of Quantization Bits')
parser.add_argument('--frac_bits', type=int, default=8, help='Number of Quantization Bits')
parser.add_argument('--bBW', type=int, default=8, help='Number of bit width')
args = parser.parse_args(args=[])

## Definition

### int2bin : int number to binary number

In [3]:
def int2bin(iIn,iBW):
    iBW = iBW + 1
    if iIn >= 0:
        bOut = bin(iIn).replace('0b','').rjust(iBW,'0')
    else :
        bOut = bin(iIn & (pow(2,iBW)-1)).replace('0b','').rjust(iBW,'1')
    return bOut[1:]

### XOR : xor gate with string

In [4]:
def XOR(iA,iB):
    if iA != iB :
        iOut = '1'
    else : 
        iOut = '0'
    return iOut

### SNUM : sign number determination

In [5]:
def snum(a):
    if a >= 0 :
        return '0'
    else :
        return '1'

## bInv : Binary Inversion

In [6]:
def binInv(bIn):
    bOut = bin(int(bIn,2)^(pow(2,len(bIn))-1)).replace('0b','').rjust(len(bIn),'0')
    return bOut

### fxp & flp2fix : floating number to fixed number conversion

In [7]:
class fxp:
    def __init__(self, bIn, iBWF):
        self.iFullBW = len(bIn)
        self.iIntgBW = self.iFullBW - iBWF
        self.bSign = bIn[0]
        self.bIntg = bIn[:self.iIntgBW]
        self.bFrac = bIn[self.iIntgBW:]
        self.fFull = 0
        try:
            for idx, bit in enumerate(bIn):
                if idx == 0:
                    self.fFull = self.fFull + int(bit,2) * -pow(2, self.iIntgBW - 1)
                else:
                    self.fFull = self.fFull + int(bit,2) * pow(2, self.iIntgBW - 1 - idx)
        except:
            print(bIn)
        self.dispFull = self.bIntg +"."+ self.bFrac 
        return

In [8]:
class flp2fix:
    def __init__(self, fIn, iBW, iBWF):
        self.fMin = - 2 ** (iBW - iBWF - 1)
        self.fMax = (2 ** (iBW-1) - 1) * (2 ** -iBWF)
        self.fResol = 2 ** -iBWF
        if fIn < self.fMin or fIn > self.fMax:
            print(f'({fIn}): Out of input range ({self.fMax}/{self.fMin}) during flp -> fix converting ')
        self.iBW = iBW
        self.iBWI = iBW - iBWF
        self.iBWF = iBWF

        self.iFLP2INT = abs(int(fIn * 2 ** iBWF))
        if fIn < 0:
            self.iFLP2INT = 2 ** (iBW-1) - self.iFLP2INT

        if fIn >= 0:
            self.bFull = bin(self.iFLP2INT)[2:].rjust(iBW, '0')
        else:
            self.bFull = '1'+bin(self.iFLP2INT)[2:].rjust(iBW-1, '0')
            if len(self.bFull) > iBW:
                self.bFull = '0' * iBW

        self.cssFxp = fxp(self.bFull, self.iBWF)
        self.bSign = self.cssFxp.bSign
        self.bIntg = self.cssFxp.bIntg
        self.bFrac = self.cssFxp.bFrac
        self.fFull = self.cssFxp.fFull
        return

### LFSR : make pseudorandom number bitstream

In [9]:
class LFSR8:
    def Random(self):
        self.b0 = eval(f'str(random.randint(0,1))')
        self.b1 = eval(f'str(random.randint(0,1))')
        self.b2 = eval(f'str(random.randint(0,1))')
        self.b3 = eval(f'str(random.randint(0,1))')
        self.b4 = eval(f'str(random.randint(0,1))')
        self.b5 = eval(f'str(random.randint(0,1))')
        self.b6 = eval(f'str(random.randint(0,1))')
        self.b7 = eval(f'str(random.randint(0,1))')
        return self.b0 + self.b1 + self.b2 + self.b3 + self.b4 + self.b5 + self.b6 + self.b7
    
    def Normal(self,stream):
        self.b0 = XOR(int(XOR(int(stream[5]),int(XOR(int(stream[3]),int(stream[4]))))),int(stream[7]))
        self.b1 = stream[0]
        self.b2 = stream[1]
        self.b3 = stream[2]
        self.b4 = stream[3]
        self.b5 = stream[4]
        self.b6 = stream[5]
        self.b7 = stream[6]
        
        return self.b0 + self.b1 + self.b2 + self.b3 + self.b4 + self.b5 + self.b6 + self.b7
    
    def Allzero(self):
        self.b0 = '0'
        self.b1 = '0'
        self.b2 = '0'
        self.b3 = '0'
        self.b4 = '0'
        self.b5 = '0'
        self.b6 = '0'
        self.b7 = '0'
        
        return self.b0 + self.b1 + self.b2 + self.b3 + self.b4 + self.b5 + self.b6 + self.b7

### LFSRlist : make pseudorandom number bitstream with 2**bBW cycle

In [10]:
def LFSRlist8():
    lfsr = LFSR8()
    lfsrlist = []
    for k in range(2**(8)-1): #lfsr number generating
        if k == 0:
            lfsrlist.append(lfsr.Random())
        else :
            lfsrlist.append(lfsr.Normal(lfsrlist[k-1]))
        if (k == 2**(8)-2):
            lfsrlist.append(lfsr.Allzero())
    
    if (args.bBW) != args.frac_bits :
        if args.bBW < args.frac_bits :
            for i in range(len(lfsrlist)):
                lfsrlist[i] = lfsrlist[i] + (args.frac_bits-args.bBW)*'0'
        else :
            print("it can't work")
            return 0
    
    return lfsrlist

### Comp : Comparator in SNG

In [11]:
def Comp(a,lfsr,snum):
    for com in range(0,len(a)):
        oA = '0'
        if a[com]!=lfsr[com]:
            if(int(a[com]) > int(lfsr[com])):
                oA = '1'
            break
    return XOR(oA,snum)

### Perm : module for permutation SNG 

In [12]:
def perm(a):
    al = len(a)
    blist = []
    for i in range(al) :
        #print(al-i-1)
        blist.append(a[al-i-1])
    
    b = "".join(blist)
    
    return b

### findActMaxMin : find abs max value in act tensor

In [13]:
def findActMaxMin():
    SF = {}
    for i in range(6):
            max = torch.max(eval(f'act{i}'))
            min = torch.min(eval(f'act{i}'))
            exec(f"SF['act{i}']=torch.max(abs(max),abs(min)).item()")
    return SF

### findWeiMaxMin : find abs max balue in weight tensor --> it will be changed in MLP module

In [14]:
def findWeiMaxMin():
    SF = {}
    for i in range(6):
            max = torch.max(eval(f'wei{i}'))
            min = torch.min(eval(f'wei{i}'))
            exec(f"SF['wei{i}']=torch.max(abs(max),abs(min)).item()")
    return SF

### SNG : stochastic number generator module

In [15]:
def SNG(iIN,lfsr):

    sNUM = snum(iIN)
    
    bIN = flp2fix(iIN,args.full_bits,args.frac_bits).bFull
    bFRAC = bIN[-(args.frac_bits):]
    if sNUM == 1 :
        bFRAC = bin(int(binInv(bFRAC),2)+1).replace('0b','').rjust(args.bBW,'0')
    oAlist = []
    
    for k in range(2**(args.bBW)): #lfsr number generating
        lNUM = lfsr[k]
        a = Comp(bFRAC,lNUM,sNUM)
        oAlist.append(a) #comparator of input a
    
    oAlist.insert(0,sNUM)
    sA = "".join(oAlist)
    if bIN == '0'*args.full_bits :
        return '0'*args.bBW
    else :
        return sA

### SNG_P : permutation stochastic number generator module

In [16]:
def SNG_P(iIN,lfsr):
    sNUM = snum(iIN)
    
    bIN = flp2fix(iIN,args.full_bits,args.frac_bits).bFull
    bFRAC = bIN[-(args.frac_bits):]
    if sNUM == 1 :
        bFRAC = bin(int(binInv(bFRAC),2)+1).replace('0b','').rjust(args.bBW,'0')
    oAlist = []
    
    for k in range(2**(args.bBW)): #lfsr number generating
        if (args.bBW == args.frac_bits) :    
            lNUM = perm(lfsr[k])
        elif (args.bBW < args.frac_bits) :
            lNUM = perm(lfsr[k][:args.bBW])+(args.frac_bits-args.bBW)*"0"
        a = Comp(bFRAC,lNUM,sNUM)
        oAlist.append(a) #comparator of input a
    
    oAlist.insert(0,sNUM)
    sA = "".join(oAlist)
    if bIN == '0'*args.full_bits :
        return '0'*args.bBW
    else :
        return sA

### SNGnumpy : SNG module with numpy

In [17]:
def SNGnumpy(fIn,lfsr):
    sList = []
    for aNumpy in fIn.view(-1):
        sList.append(SNG(float(aNumpy),lfsr))
                     
    return np.array(sList).reshape(fIn.size())

### SNGpnumpy : permutation SNG module with numpy

In [18]:
def SNGpnumpy(fIn,lfsr):
    sList = []
    for aNumpy in fIn.view(-1):
        sList.append(SNG_P(float(aNumpy),lfsr))
                     
    return  np.array(sList).reshape(fIn.size())

### CountOne : count 1 in stochastic number bit stream

In [19]:
def CountOne(nIn):
    nlist = []
    for num in nIn.reshape(-1):
        n = 0
        for a in num:
            if a == '1' :
                n += 1
        if a[0] == '1' :
            nlist.append(n-1)
        else :
            nlist.append(n)
    return torch.tensor(nlist).view(nIn.shape)

### defSign : sign determination in S2N 

In [20]:
def defSign(nIn):
    nlist = []
    for num in nIn.reshape(-1):
        if num[0] == '1' :
            nlist.append(-1)
        else :
            nlist.append(1)
    return torch.tensor(nlist).view(nIn.shape)

### Multiplier

In [21]:
def mul(a,b):
    al = len(a)
    bl = len(b)
    
    outlist = []
    
    if al != bl :
        print("length of string is different")
        return 0
    
    outlist.append(XOR(a[0],b[0]))
    
    for i in range(al-1) :
        outlist.append(str(int(a[i+1]) & int(b[i+1])))
    
    #print(outlist)
    out = "".join(outlist)
    
    return out    

In [22]:
def defSign1(nIn):
    if nIn[0] == '1' :
        return -1
    else :
        return 1

In [23]:
def CountOne1(nIn):
    n = 0
    for num in nIn:
        if num == '1' :
            n += 1
    if nIn[0] == '1' :
        return n-1
    else :
        return n

In [24]:
def S2None(sIn,SF):
    s = defSign1(sIn)
    o = (CountOne1(sIn)/(2**args.bBW))*SF*s
    return o

In [25]:
def mulNumpy(aIn,bIn,aSF,wSF):
    mList = []
    for i in range(aIn.shape[0]):
        for j in range(bIn.T.shape[1]):
            sum = 0
            for k in range(aIn.shape[1]):
                sum += S2None(mul((aIn[i][k].astype(str)),(bIn.T)[k][j].astype(str)),aSF*wSF)
            mList.append(sum)
    return torch.tensor(mList).view(aIn.shape[0],bIn.T.shape[1])

## TEST

In [26]:
act0 = torch.rand((9,9))-torch.randint(4,(9,9))+torch.randint(4,(9,9))
act1 = torch.rand((9,9))-torch.randint(4,(9,9))+torch.randint(4,(9,9))
act2 = torch.rand((9,9))-torch.randint(4,(9,9))+torch.randint(4,(9,9))
act3 = torch.rand((9,9))-torch.randint(4,(9,9))+torch.randint(4,(9,9))
act4 = torch.rand((9,9))-torch.randint(4,(9,9))+torch.randint(4,(9,9))
act5 = torch.rand((9,9))-torch.randint(4,(9,9))+torch.randint(4,(9,9))

wei0 = torch.rand((9,9))-torch.randint(4,(9,9))+torch.randint(4,(9,9))
wei1 = torch.rand((9,9))-torch.randint(4,(9,9))+torch.randint(4,(9,9))
wei2 = torch.rand((9,9))-torch.randint(4,(9,9))+torch.randint(4,(9,9))
wei3 = torch.rand((9,9))-torch.randint(4,(9,9))+torch.randint(4,(9,9))
wei4 = torch.rand((9,9))-torch.randint(4,(9,9))+torch.randint(4,(9,9))
wei5 = torch.rand((9,9))-torch.randint(4,(9,9))+torch.randint(4,(9,9))

### number to stochastic conversion

In [27]:
def N2S(act0,act1,act2,act3,act4,act5,wei0,wei1,wei2,wei3,wei4,wei5):
    aSF = findActMaxMin()
    wSF = findWeiMaxMin()
    
    lfsrlist0 = LFSRlist8()
    lfsrlist1 = LFSRlist8()
    lfsrlist2 = LFSRlist8()
    lfsrlist3 = LFSRlist8()
    lfsrlist4 = LFSRlist8()
    lfsrlist5 = LFSRlist8()
    
    #LFSR error checking
    for i in range(6):
            exec(f'tt_lfsrlist{i} = dict.fromkeys(lfsrlist{i})')
            exec(f't_lfsrlist{i} = list(tt_lfsrlist{i})')
            exec(f"if(lfsrlist{i} != t_lfsrlist{i}): print('lfsrlist{i} is error')")
    
    
    a0 = SNGnumpy(act0/aSF['act0'],lfsrlist0)
    a1 = SNGnumpy(act1/aSF['act1'],lfsrlist1)
    a2 = SNGnumpy(act2/aSF['act2'],lfsrlist2)
    a3 = SNGnumpy(act3/aSF['act3'],lfsrlist3)
    a4 = SNGnumpy(act4/aSF['act4'],lfsrlist4)
    a5 = SNGnumpy(act5/aSF['act5'],lfsrlist5)
    

    w0 = SNGpnumpy(wei0/wSF['wei0'],lfsrlist0)
    w1 = SNGpnumpy(wei1/wSF['wei1'],lfsrlist1)
    w2 = SNGpnumpy(wei2/wSF['wei2'],lfsrlist2)
    w3 = SNGpnumpy(wei3/wSF['wei3'],lfsrlist3)
    w4 = SNGpnumpy(wei4/wSF['wei4'],lfsrlist4)
    w5 = SNGpnumpy(wei5/wSF['wei5'],lfsrlist5)
    
    return a0,a1,a2,a3,a4,a5,w0,w1,w2,w3,w4,w5

### stochastic to number conversion

In [28]:
def S2N(a0,a1,a2,a3,a4,a5,w0,w1,w2,w3,w4,w5):
    aSF = findActMaxMin()
    wSF = findWeiMaxMin()
    
    sa0 = defSign(a0)
    sa1 = defSign(a1)
    sa2 = defSign(a2)
    sa3 = defSign(a3)
    sa4 = defSign(a4)
    sa5 = defSign(a5)
    
    sw0 = defSign(w0)
    sw1 = defSign(w1)
    sw2 = defSign(w2)
    sw3 = defSign(w3)
    sw4 = defSign(w4)
    sw5 = defSign(w5)
    
    ca0 = (CountOne(a0)/(2**args.bBW)) * aSF['act0'] *sa0
    ca1 = (CountOne(a1)/(2**args.bBW)) * aSF['act1'] *sa1
    ca2 = (CountOne(a2)/(2**args.bBW)) * aSF['act2'] *sa2
    ca3 = (CountOne(a3)/(2**args.bBW)) * aSF['act3'] *sa3
    ca4 = (CountOne(a4)/(2**args.bBW)) * aSF['act4'] *sa4
    ca5 = (CountOne(a5)/(2**args.bBW)) * aSF['act5'] *sa5
    
    cw0 = (CountOne(w0)/(2**args.bBW)) * wSF['wei0'] *sw0
    cw1 = (CountOne(w1)/(2**args.bBW)) * wSF['wei1'] *sw1
    cw2 = (CountOne(w2)/(2**args.bBW)) * wSF['wei2'] *sw2
    cw3 = (CountOne(w3)/(2**args.bBW)) * wSF['wei3'] *sw3
    cw4 = (CountOne(w4)/(2**args.bBW)) * wSF['wei4'] *sw4
    cw5 = (CountOne(w5)/(2**args.bBW)) * wSF['wei5'] *sw5

    return ca0,ca1,ca2,ca3,ca4,ca5,cw0,cw1,cw2,cw3,cw4,cw5

In [29]:
a0,a1,a2,a3,a4,a5,w0,w1,w2,w3,w4,w5 = N2S(act0,act1,act2,act3,act4,act5,wei0,wei1,wei2,wei3,wei4,wei5)

In [30]:
ca0,ca1,ca2,ca3,ca4,ca5,cw0,cw1,cw2,cw3,cw4,cw5 = S2N(a0,a1,a2,a3,a4,a5,w0,w1,w2,w3,w4,w5)

In [31]:
def FindError(a,b):
    error = 0
    for anum,bnum in zip(a.view(-1),b.view(-1)):
        e = (abs(bnum-anum)*100).item()
        error += e
    return error/(len(a.view(-1))+len(b.view(-1)))

In [32]:
ea0 = FindError(act0,ca0)
ea1 = FindError(act1,ca1)
ea2 = FindError(act2,ca2)
ea3 = FindError(act3,ca3)
ea4 = FindError(act4,ca4)
ea5 = FindError(act5,ca5)


ew0 = FindError(wei0,cw0)
ew1 = FindError(wei1,cw1)
ew2 = FindError(wei2,cw2)
ew3 = FindError(wei3,cw3)
ew4 = FindError(wei4,cw4)
ew5 = FindError(wei5,cw5)

In [33]:
print(f'act0 error is {ea0}')
print(f'act1 error is {ea1}')
print(f'act2 error is {ea2}')
print(f'act3 error is {ea3}')
print(f'act4 error is {ea4}')
print(f'act5 error is {ea5}')


print(f'wei0 error is {ew0}')
print(f'wei1 error is {ew1}')
print(f'wei2 error is {ew2}')
print(f'wei3 error is {ew3}')
print(f'wei4 error is {ew4}')
print(f'wei5 error is {ew5}')

act0 error is 3.157676726579666
act1 error is 3.243170612940082
act2 error is 3.2625187484570493
act3 error is 3.366537999775675
act4 error is 3.0978059930565918
act5 error is 2.8641295951825603
wei0 error is 3.155203405905653
wei1 error is 3.261602920882496
wei2 error is 3.277868859561873
wei3 error is 3.1722645435804204
wei4 error is 3.0368728257807684
wei5 error is 3.1851526311150304


## Multiply test

In [34]:
a0, a1, a2, w0, w1, w2 = N2S(act0,act1,act2,wei0,wei1,wei2)

TypeError: N2S() missing 6 required positional arguments: 'wei0', 'wei1', 'wei2', 'wei3', 'wei4', and 'wei5'

In [ ]:
aSF = findActMaxMin()
wSF = findWeiMaxMin()

In [ ]:
def multest(aIn,wIn,aSF,wSF):
    mlist=[]
    sum=0
    for i in range(aIn.shape[1]):
        a = S2None(mul((aIn[0][i].astype(str)),(wIn.T)[i][0].astype(str)),aSF*wSF)
        print(a)
        sum += a
    mlist.append(sum)
    return torch.tensor(mlist).view(1,1)
    

In [ ]:
multest(a0,w0,aSF['act0'],wSF['wei0'])

In [ ]:
def nummultest(aIn,wIn,aSF,wSF):
    mlist=[]
    sum=0
    for i in range(aIn.shape[1]):
        a = aIn[0][i]*(wIn.T)[i][0]
        print(a)
        sum += a
    mlist.append(sum)
    return torch.tensor(mlist).view(1,1)

In [ ]:
nummultest(act0,wei0,aSF['act0'],wSF['wei0'])

In [ ]:
sout0 = mulNumpy(a0,w0,aSF['act0'],wSF['wei0'])
#sout1 = mulNumpy(a1,w1)
#sout2 = mulNumpy(a2,w2)